[View in Colaboratory](https://colab.research.google.com/github/MChamith/SpatioAE/blob/master/trainae.ipynb)

In [1]:
# !git clone https://github.com/MChamith/SpatioAE

Cloning into 'SpatioAE'...
remote: Counting objects: 226, done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 226 (delta 3), reused 0 (delta 0), pack-reused 182
Receiving objects: 100% (226/226), 6.34 MiB | 34.52 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [0]:

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Activation, BatchNormalization
from keras.models import Model
from keras import backend as K
import cv2
import numpy as np
import os
from tqdm import tqdm

TRAIN_DIR ='SpatioAE/Train001'
IMG_SIZE = 128



def create_training_set():
  train_data = []
  for img in tqdm(os.listdir(TRAIN_DIR)):
    path = os.path.join(TRAIN_DIR, img)
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE) )
    img = np.expand_dims(img, axis = 2)
    train_data.append(np.array(img))
    
    
  np.save('training_data.npy' , train_data)
  
  return train_data

In [0]:

def model_autoencoder():
  
  input_img  = Input(shape = (128,128,1))  #channel last
  
  x = Conv2D(filters = 32, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(input_img)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
  print("layer one conv " + str(x.shape))
  
  x = Conv2D(filters = 48, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
  print("layer two conv " + str(x.shape))
  
 
  x = Conv2D(filters = 64, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(pool_size = (2,2), strides = (1,1))(x)
  print("layer three conv " + str(x.shape))
  
  x = Conv2D(filters = 64, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  encoded = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
  print("layer four conv " + str(encoded.shape))
  
  
  ## decoder branch
  
  x = Conv2D(filters = 48, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(encoded)
  x = UpSampling2D(size = (2,2))(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  print("layer one deconv " + str(x.shape))
 
  x = Conv2D(filters = 32, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = UpSampling2D(size = (2,2))(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  print("layer two deconv " + str(x.shape))
  
  x = Conv2D(filters = 32, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = UpSampling2D(size = (2,2))(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  print("layer three deconv " + str(x.shape))
  decoded = Conv2D(filters = 1, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'sigmoid')(x)
  print("output" + str(decode.shape))

  
  
  autoencoder = Model(input_img, decoded)
  
  return autoencoder
  

In [82]:
get_data = create_training_set()
input_imgs = np.load('training_data.npy')
print(input_imgs.shape)
autoencoder.clear_session()
autoencoder = model_autoencoder()
autoencoder.compile(optimizer = 'adadelta' , loss = 'binary_crossentropy')

autoencoder.fit(input_imgs,input_imgs,  epochs =100)
  
    

100%|██████████| 199/199 [00:00<00:00, 2662.33it/s]


(199, 128, 128, 1)
layer one conv (?, 64, 64, 32)
layer two conv (?, 32, 32, 48)
layer three conv (?, 16, 16, 64)
layer four conv (?, 8, 8, 64)
layer one deconv (?, 16, 16, 48)
layer two deconv (?, 32, 32, 32)
layer three deconv (?, 64, 64, 32)


NameError: ignored